In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
model= ChatOpenAI(model_name="gpt-3.5-turbo")

### Lets go Step by Step

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, I am Vikas")])

AIMessage(content='Hello Vikas! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 13, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fd40e84b-6a56-46a3-9c4b-8db0697aecae-0')

* Now lets check if it remembers my name

In [4]:

model.invoke([HumanMessage(content="Whats my name?")])

AIMessage(content="I'm sorry, I do not know your name as we are communicating through text.", response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 11, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-02f7e555-a79f-4ad2-a481-1049d4a4276f-0')

* It didn't remember the name
    * Now we will save some predefined chat and ask the model if it remembers old chat

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Vikas"),
        AIMessage(content="Hello Vikas! How can i help you?"),
        HumanMessage(content="What's my name")
        
    ]
)

AIMessage(content='Your name is Vikas.', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 35, 'total_tokens': 41}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9e17f3c8-f6c8-4be1-9b9c-cce3a8f78ed7-0')

### Now it is remembering the name
* So Chatbot has to have previous messages to remember what we have told
#### Lets Streamline this

* We will store our chat in data store and use it for future interactions by retrieving them and pass them into chain as part of input.
* We will manage this with session ids

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [7]:
store= {}
## Get session history would give you the chat history of the session or it will store it if it is new
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]= ChatMessageHistory()
    return store[session_id]

## Managing chat: Where storing and appending new chat happens here
with_message_history= RunnableWithMessageHistory(model, get_session_history)

* We will create config now that we have to pass into runnables everytime. This config consists of session id

In [8]:
config= {"configurable": {"session_id": "vikas"}}

In [9]:
response= with_message_history.invoke(
    [
        HumanMessage(content="Hello, I'm Vikas")
    ],
    config=config
)


Parent run 68329485-5440-43c5-ad7c-8c74558c307b not found for run f9be3a1c-3f32-4f84-aa6d-8469b8515b62. Treating as a root run.


In [10]:
response.content

'Nice to meet you Vikas! How can I assist you today?'

* Now lets ask my name again

In [11]:
response= with_message_history.invoke(
    [
        HumanMessage(content="What is my name")
    ], config= config
)
response.content

Parent run e11023d1-f62a-413c-8c3e-8c5ab461fcad not found for run 52cb7aff-3a3e-4044-bd35-e90529c32176. Treating as a root run.


'Your name is Vikas.'

## Cool! Now the chat remembers history

* Lets change the session id and check

In [12]:
config_2= {"configurable": {"session_id": "vicky"}}
response= with_message_history.invoke(
    [
        HumanMessage(content="What is my name?")
    ], config= config_2
)
response.content

Parent run ecc3ea4b-d41f-45a3-9177-adb221aeac5e not found for run 039c6fec-dae9-4e7f-9618-8b0c05b54b5d. Treating as a root run.


"I'm sorry, I do not have the ability to know your name."

* Now lets goback and ask

In [13]:

respose= with_message_history.invoke(
    [
        HumanMessage(content="What is my name?")
        
    ],
    config= config
)
response.content

Parent run 7baa400c-2299-4b3a-b5a1-7ee48c8230d4 not found for run 17602afe-9fc5-439a-9ed1-f74c3e45d5f2. Treating as a root run.


"I'm sorry, I do not have the ability to know your name."

## This is very basic layer of the chat history
### Lets dig more into it and understand
* We will find a way to attach history to prompt template as well

In [15]:
template = """
You are creating a Certificate for an employee. Please provide the following details:

Employment Resignation:
This certificate generates resignation details of employee. If language is German then Formular id= ARB_AUS else Formular id= ARB_AUS_EN and generates certificate provided below information:
1. Employee Id: 
2. Bv Number:
3. Effective Date (YYYY-MM-DD):
4. Language (English or German): 

If all the necessary values are provided then summarize the inputs and ask for confirmation.
If user confirms with a confirmation message then convert the details in url format with prefix https://certificates/certificate name/certificate details seperated by '/'.


Employment details:
This certificate generates employment details of employee requested. If language is German then Formular id= ARB_ESS else Formular id= ARB_ESS_EN and generates certificate provided below information:
1. Employee Id: 
2. Bv Number:
3. Effective Date (YYYY-MM-DD):
4. Language (English or German): 

If all the necessary values are provided then summarize the inputs and ask for confirmation.
If user confirms with a confirmation message then convert the details in url format with prefix https://certificates/certificate name/certificate details seperated by '/'.

If you have any questions or need clarification, feel free to ask!

"""

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # "you are a helpful assistant. Answer all the questions to the best of your ability."
            template
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain= prompt | model

In [17]:
response= chain.invoke(
    {
        "messages": [
            HumanMessage(content="Hi! I'm vikas")
        ]
    }
)
response.content


'Hello, Vikas! How can I assist you today?'

### Now lets add message history to the chat
* We will create Runnablewithhistory but instead of model we will pass chain as parameter

In [18]:
with_message_history= RunnableWithMessageHistory(chain, get_session_history)
config= {"configurable": {"session_id":"vikas"}}

In [ ]:
template

In [19]:
response= with_message_history.invoke(
    [
        HumanMessage(content="Can you create certificates?")
    ],
    config= config
)

# response.content
response

Parent run 90a27428-9f5f-47c6-b0a7-0d28f0b3c98f not found for run 96b5ba5c-f749-462b-8a6a-9ec3a2124167. Treating as a root run.


AIMessage(content='Yes, I can help you create certificates. What kind of certificate do you need? An employment resignation certificate or an employment details certificate?', response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 337, 'total_tokens': 364}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-16592ea4-cf95-4e96-9a8e-3081507e41e7-0')

In [20]:
response= with_message_history.invoke(
    [
        HumanMessage(content="What is my name")
    ],
    config= config
)

response.content

Parent run 632474f5-938e-4b81-8aef-b7b7f7ad6847 not found for run 93eccace-2e67-48e3-bebf-30687bb1425d. Treating as a root run.


'Your name is Vikas. How can I assist you today, Vikas?'

## Super! Lets add few other parameters such as respond in different language

In [ ]:
prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all question tp best of your ability in {language}."
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain= prompt| model

In [ ]:
response= chain.invoke(
    {"messages":[HumanMessage(content="Hello, I'm Vikas")], "language": "Spanish"
        }
)
response.content

In [ ]:
## Create runnable char history
with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [ ]:
config= {"configurable": {"session_id":"vikas"}}

In [ ]:
response= with_message_history.invoke(
    {"messages":[HumanMessage(content="Hello, I'm Vikas")], "language": "Spanish"
        }, config=config
)
response.content

In [ ]:
response= with_message_history.invoke(
    {
        "messages": [
            HumanMessage(content="What is my name?")
        ], "language":"Spanish"
    }, config=config
)
response.content

### Managing Chat history
* We can add chat history like above. But what if storing goes on for all the chat. 
    * Memory issue
    * Overflowing of the context window of LLM
* So We will add a step to limit the size of the messages that we are passing in
* This has to be done Before the prompt template, but After loading messages from message history.
* We can do this by creating a function to select k most recent messages.

In [ ]:
from langchain_core.runnables import RunnablePassthrough

def filter_messages(messages, k=10):
    return messages[-k:]

chain= (
    RunnablePassthrough.assign(messages= lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [ ]:
messages = [
    HumanMessage(content="hi! I'm Vikas"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like butterscotch ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [ ]:
response =  chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What is my name")],
        "language":"English"
    }
)
response.content

### This is because The limit is 10 and the entered messages is 11th so it left the first message

In [ ]:
response =  chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What is my Fav ice Cream")],
        "language":"English"
    }
)
response.content

* The response was within the chat

#### Lets Wrap with message history with session id

In [ ]:
with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config= {"configurable": {"session_id": "vikas"}}

In [ ]:
response= with_message_history.invoke(
    {
        "messages": [
            HumanMessage(content="What is my name?")
        ], "language":"english"
    }, config= config
)
response.content

In [ ]:
response= with_message_history.invoke(
    {
        "messages": [
            HumanMessage(content="What is my favourite ice cream?")
        ], "language":"english"
    }, config= config
)
response.content

In [ ]:
response= with_message_history.invoke(
    {
        "messages": [
            HumanMessage(content="It is butter scotch?")
        ], "language":"english"
    }, config= config
)
response.content

In [ ]:
response= with_message_history.invoke(
    {
        "messages": [
            HumanMessage(content="Can you tell me now, what is my favourite ice cream?")
        ], "language":"english"
    }, config= config
)
response.content

In [ ]:
from pprint import pprint
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    pprint(r.content)

In [21]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAI
### Packages which handle Chat history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
##Chat prompt Template
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model_name='gpt-3.5-turbo')
## Define LLLM congifuration
template = """
You are creating a Certificate of Employment for an employee. Please provide the following details:

Employment Resignation:
This certificate generates resignation details of employee. If language is German then Formular id= ARB_AUS else Formular id= ARB_AUS_EN and generates certificate provided below information:
1. Employee Id: 
2. Bv Number:
3. Effective Date (YYYY-MM-DD):
4. Language (English or German): 

If all the necessary values are provided then covert the result in url format with prefix https://certificates/certificate name/certificate details seperated by '/'.


Employment details:
This certificate generates employment details of employee requested. If language is German then Formular id= ARB_ESS else Formular id= ARB_ESS_EN and generates certificate provided below information:
1. Employee Id: 
2. Bv Number:
3. Effective Date (YYYY-MM-DD):
4. Language (English or German): 

Please provide each detail one by one in the given order. If you have any questions or need clarification, feel free to ask!

"""
# Now we can override it and set it to "Friend"
# from langchain_core.prompts.prompt import PromptTemplate

prompt =    ChatPromptTemplate.from_messages( [
        (
            "system",
            template
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
### Lets Create Chain
chain= prompt | llm

### Chat message history
## Function that stores chat based on session id
store= {}
## Get session history would give you the chat history of the session or it will store it if it is new
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]= ChatMessageHistory()
    return store[session_id]

with_message_history= RunnableWithMessageHistory(
    chain, get_session_history,
       
)
## Create session id configuration
config= {"configurable": {"session_id":"vikas"}}


In [22]:
input_text= "Hi i am Vikas"

In [24]:
def chat_response(input_text: str) -> str:
    return with_message_history.invoke(
    {
        "message": HumanMessage(content= input_text)
    }, config= config
).content

In [23]:
response= with_message_history.invoke(
    {
        "message": HumanMessage(content= input_text)
    }, config= config
)
print(response.content)

Parent run 4aee4e7c-227d-4b94-85a2-2c1e1659d615 not found for run 97cd78fa-6192-4c34-893c-813952a6206a. Treating as a root run.


Hello Vikas! How can I assist you today?


In [25]:
chat_response("What is my name?")

Parent run 7392384e-2dc7-49eb-836a-4862f8252118 not found for run 295d6677-2374-4e4e-b5a3-b311b5001663. Treating as a root run.


'Your name is Vikas. How can I assist you further, Vikas?'

In [31]:
%pprint

Pretty printing has been turned ON


In [33]:
print(chat_response("Can you Generate me a certificate?"))

Parent run 5c02c2c6-2332-4f79-96fb-1aee48fc1baf not found for run 357c9cfe-0e8a-40eb-9e3e-9bebc9ddc5eb. Treating as a root run.


Sure, I can help with that! 

Please provide me with the following details for the certificate you need:

1. Employee Id:
2. Bv Number:
3. Effective Date (YYYY-MM-DD):
4. Language (English or German):


In [34]:
print(chat_response("How many certificates are available?"))

Parent run 8bf364ea-284b-4ebf-9652-e334fcf3bec7 not found for run 11ad99fa-801a-42d1-98db-7c9d36fad9bb. Treating as a root run.


There are two types of certificates available:

1. Certificate of Employment Resignation
2. Certificate of Employment Details

Please let me know which one you would like to generate and provide the necessary details accordingly.


In [35]:
print(chat_response("Employment Resignation"))

Parent run 2c60c26d-5301-40dc-ad6d-fb4087e428fb not found for run b37f33d1-6c8c-498a-bd58-3e0b1659a057. Treating as a root run.


Great choice! Please provide me with the following details for the Certificate of Employment Resignation:

1. Employee Id:
2. Bv Number:
3. Effective Date (YYYY-MM-DD):
4. Language (English or German):


In [36]:
print(chat_response("Which certificate did i ask for?"))

Parent run b0cc828f-6f86-473a-819d-fe06b3bb8a0d not found for run d21538cd-27a2-49e4-9f4c-79a04b6c1972. Treating as a root run.


You asked for the Certificate of Employment Resignation. Let me know if you would like to proceed with generating that certificate now.


In [37]:
print(chat_response("Yes, Please proceed with that"))

Parent run ceb582ec-83b0-4cac-817c-622a9f147256 not found for run 36c6fd34-fb3e-4dbd-8999-2ea4f8c5afc0. Treating as a root run.


Great! Please provide me with the necessary details for the Certificate of Employment Resignation:

1. Employee Id:
2. Bv Number:
3. Effective Date (YYYY-MM-DD):
4. Language (English or German):


In [38]:
print(chat_response("1. Employee Id:123, 2. Bv Number:12, 3. Effective Date (YYYY-MM-DD):34, 4. Language (English or German): German"))

Parent run 4c2478e1-75cc-4dac-bab8-09ca1b8d01d3 not found for run 46fbbae8-586e-46f4-bf52-6cddb5cc5422. Treating as a root run.


Thank you for providing the details. Let me generate the Certificate of Employment Resignation for you in German.

Certificate details:
- Employee Id: 123
- Bv Number: 12
- Effective Date: 34
- Language: German

The Formular id for German language is ARB_AUS.

Here is the URL for your Certificate of Employment Resignation in German:
https://certificates/certificate%20of%20employment%20resignation/123/12/34/German/ARB_AUS


In [39]:
print(chat_response("Thank you"))

Parent run fb401b90-2dd9-41a5-9f99-e40b57cbe28e not found for run 0851926f-a144-45fd-a8b9-48c49db80ba1. Treating as a root run.


You're welcome! If you need any further assistance or more certificates in the future, feel free to ask. Have a great day!


In [45]:
store={}

In [46]:
print(chat_response("What is my name?"))

Parent run 48e05a80-afcd-4671-87d5-b0aa0588c56d not found for run fe055604-f902-4d26-80d7-ce67d81517ba. Treating as a root run.


I'm here to help you with creating a Certificate of Employment for an employee. Please provide the necessary details for either the Employment Resignation or Employment details certificate. If you have any questions about the process, feel free to ask!


In [ ]:

template = """
You are creating a Certificate of Employment for an employee. Please provide the following details:

Employment Resignation:
This certificate generates resignation details of employee. If language is German then Formular id= ARB_AUS else Formular id= ARB_AUS_EN and generates certificate provided below information:
1. Employee Id: 
2. Bv Number:
3. Effective Date (YYYY-MM-DD):
4. Language (English or German): 

If all the necessary values are provided then covert the result in url format with prefix https://certificates/certificate name/certificate details seperated by '/'.


Employment details:
This certificate generates employment details of employee requested. If language is German then Formular id= ARB_ESS else Formular id= ARB_ESS_EN and generates certificate provided below information:
1. Employee Id: 
2. Bv Number:
3. Effective Date (YYYY-MM-DD):
4. Language (English or German): 

Please provide each detail one by one in the given order. If you have any questions or need clarification, feel free to ask!
"""

chat_prompt_template = ChatPromptTemplate.from_template(template)
